In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2
import os
import pathlib
import sqlalchemy as sa
import sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

from tables import County, DataTrustedIdentifier, InfoImage, create_info_image
from database import connect, state_province_in_list_uf_or_list_state, has_state_in_locality
from sqlalchemy import and_, or_
from sqlalchemy.orm import Session

from unaccent import unaccent

def text_bold(string):
    return '\033[1m' + string + '\033[0m'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## database connection

In [2]:
session: Session
engine, session = connect()
engine.echo=False

2023-03-19 01:13:40,240 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-19 01:13:40,241 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-19 01:13:40,243 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-19 01:13:40,243 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-19 01:13:40,245 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-19 01:13:40,245 INFO sqlalchemy.engine.Engine [raw sql] {}


In [ ]:
def set_path_image(color_mode, image_size, list_images, query):
    for i, q in enumerate(query):
        barcode = q[0]
        seq = q[1]
        result = sorted([str(image) for image in list_images if barcode in str(image.stem) and os.path.exists(image)])
        print('founded %d files' % len(result))
        if len(result) > 0:
            for r in result:
                session.query(InfoImage)\
                    .filter(and_(InfoImage.color_mode==color_mode,
                                 InfoImage.image_size==image_size,
                                 InfoImage.path_image,
                                 InfoImage.seq_id==seq))\
                    .all()
                image_info = create_info_image(color_mode, image_size, r, seq)
                session.add(image_info)
                session.commit()
                session.flush()

In [ ]:
for color in ['rgb', 'grayscale']:
    for image_size in ['256', '400', '512']:
        column_name = 'path_%s_%s' % (image_size, color)
        path_fotos = '/home/xandao/Documentos/%s/%s/w_pred_mask' % (color.upper(), image_size)
        list_images = [image for image in pathlib.Path(path_fotos).rglob('*.jpeg') if image.is_file()]
        columns = [DataTrustedIdentifier.barcode, DataTrustedIdentifier.seq]
        query = session.query(*columns).all()
        print('count of images in directory %s: %d' % (path_fotos, len(query)))
        if len(list_images) > 0 and len(query) >= 0:
            set_path_image(color, image_size, list_images, query)

## BR dataset

In [3]:
county = session.query(County).distinct().all()

list_uf = [unaccent(sa.func.lower(c.uf)) for c in county]
list_state = [unaccent(sa.func.lower(c.state)) for c in county]
list_county = [unaccent(sa.func.lower(c.county)) for c in county]

query_lower_unaccent_like = session.query(sa.func.concat('%', unaccent(sa.func.lower(County.state)), '%')).distinct().all()
list_state_like = [q[0] for q in query_lower_unaccent_like]
columns = [DataTrustedIdentifier.specific_epithet, DataTrustedIdentifier.barcode]

/tmp/ipykernel_393996/1285570762.py:7: SAWarning: Class unaccent will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/20/cprf)
  query_lower_unaccent_like = session.query(sa.func.concat('%', unaccent(sa.func.lower(County.state)), '%')).distinct().all()


In [4]:
query_only_state = session.query(*columns)\
    .filter(and_(DataTrustedIdentifier.country_trusted =='Brasil',
                DataTrustedIdentifier.specific_epithet.is_not(None),
                state_province_in_list_uf_or_list_state(list_state, list_uf)))\
    .distinct()\
    .all()

print('quantidade de registros usando somente %s: %d' % (text_bold('state_province'), len(query_only_state)))

quantidade de registros usando somente state_province: 11914


In [5]:
query_has_state_in_locality = session.query(*columns)\
    .filter(and_(DataTrustedIdentifier.country_trusted=='Brasil',
                 DataTrustedIdentifier.specific_epithet.is_not(None),
                 or_(state_province_in_list_uf_or_list_state(list_state, list_uf),
                     has_state_in_locality(list_state_like))))\
    .distinct()\
    .all()

print('quantidade de registros usando somente %s: %d' % (text_bold('state_province e locality'), len(query_has_state_in_locality)))

quantidade de registros usando somente state_province e locality: 11927


In [6]:
session.commit()
session.flush()

nc = [DataTrustedIdentifier.specific_epithet, sa.func.array_agg(DataTrustedIdentifier.seq)]

qz = session.query(*nc)\
    .filter(and_(DataTrustedIdentifier.country_trusted =='Brasil',
                DataTrustedIdentifier.specific_epithet.is_not(None),
                state_province_in_list_uf_or_list_state(list_state, list_uf)))\
    .distinct()\
    .group_by(DataTrustedIdentifier.specific_epithet)\
    .having(sa.func.count(DataTrustedIdentifier.specific_epithet) >= 5)\
    .all()

len(qz)

247

In [96]:
import pandas as pd

list_images_removed = {
    'barcode': ['INPA0248526', 'INPA0248523', 'INPA0248528', 'NY01421575_01', 'HUFSJ001689_v00', 'HUFSJ001133_v00', 'HUFSJ002198_v00', 'HUFSJ003255_v00', 'HVASF000487_v01', 'INPA0019084_nd', 'INPA0022379_nd', 'INPA0032742_nd', 'INPA0023115', 'NL-U1484137', 'INPA0012286', 'INPA0146998'],
    'reason': ['horizontal', 'horizontal', 'horizontal', 'horizontal', 'not exsicate', 'not exsicate', 'not exsicate', 'not exsicate', 'not exsicate', 'label', 'label', 'label', 'letter', 'letter', 'letter', 'incomplete']
}


pd.DataFrame(list_images_removed)
# display(len(list_images_removed['barcode']), len(list_images_removed['reason']))

,barcode,reason
0,INPA0248526,horizontal
1,INPA0248523,horizontal
2,INPA0248528,horizontal
3,NY01421575_01,horizontal
4,HUFSJ001689_v00,not exsicate
5,HUFSJ001133_v00,not exsicate
6,HUFSJ002198_v00,not exsicate
7,HUFSJ003255_v00,not exsicate
8,HVASF000487_v01,not exsicate
9,INPA0019084_nd,label


In [95]:
import itertools
import numpy as np
import pandas as pd

l = []
l_species = []
paths = []
rs = []
sum = 0
for i, qq in enumerate(qz):
    species = qq[0]
    list_seq = qq[1]
    qzinho = session.query(sa.func.array_agg(sa.distinct(InfoImage.path_image))) \
        .filter(and_(InfoImage.seq_id.in_(list_seq),
                     InfoImage.image_size == image_size,
                     InfoImage.color_mode == color)) \
        .group_by(InfoImage.seq_id) \
        .all()
    lsa = list(itertools.chain(*qzinho))
    if len(lsa) >= 5:
        lb = []
        # remove images duplicates
        for lx in lsa:
            lb.append(sorted(np.unique(lx))[0])
        #check if after remove exists more than five images
        if len(np.unique(lb)) >= 5:
            l_species.append(species)
            paths.append(np.unique(lb).tolist())
            print(species, len(lb))
            sum += len(lb)
sum

[['/home/xandao/Documentos/RGB/512/w_pred_mask/MBM084431.jpeg'], ['/home/xandao/Documentos/RGB/512/w_pred_mask/MBM033212.jpeg'], ['/home/xandao/Documentos/RGB/512/w_pred_mask/MBM033181_1.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/MBM033181.jpeg'], ['/home/xandao/Documentos/RGB/512/w_pred_mask/CEN00051607_1.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/CEN00051607.jpeg'], ['/home/xandao/Documentos/RGB/512/w_pred_mask/SP011164.jpeg'], ['/home/xandao/Documentos/RGB/512/w_pred_mask/NY00555584_01.jpeg'], ['/home/xandao/Documentos/RGB/512/w_pred_mask/NY00555583_01.jpeg'], ['/home/xandao/Documentos/RGB/512/w_pred_mask/INPA0124667_e.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/INPA0124667.jpeg']]
=========+
['/home/xandao/Documentos/RGB/512/w_pred_mask/MBM084431.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/MBM033212.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/MBM033181.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/CEN00051607.jpeg', '/home/xandao/Docu

0

In [86]:
sum = 0
for pqwq in paths:
    sum += len(pqwq)
len(l_species), sum

(235, 10528)

In [87]:
sum = 0
for pqwq in paths:
    if len(pqwq) >= 5:
        sum += 1
sum

235

In [88]:
pp = {
    'species': l_species,
    'paths': paths
}

pd.DataFrame(pp)

,species,paths
0,abutiloides,[/home/xandao/Documentos/RGB/512/w_pred_mask/C...
1,aduncum,[/home/xandao/Documentos/RGB/512/w_pred_mask/A...
2,aequale,[/home/xandao/Documentos/RGB/512/w_pred_mask/C...
3,alata,[/home/xandao/Documentos/RGB/512/w_pred_mask/C...
4,alatabaccum,[/home/xandao/Documentos/RGB/512/w_pred_mask/I...
...,...,...
230,viminifolium,[/home/xandao/Documentos/RGB/512/w_pred_mask/C...
231,vitaceum,[/home/xandao/Documentos/RGB/512/w_pred_mask/I...
232,wachenheimii,[/home/xandao/Documentos/RGB/512/w_pred_mask/N...
233,warmingii,[/home/xandao/Documentos/RGB/512/w_pred_mask/M...


In [91]:
import re
# INPA0012286
l_final = []

sma = 0
for s in pp['paths']:
    matching = [sx for sx in s if not any(xs in sx for xs in list_images_removed['barcode'])]
    sma += len(matching)
    l_final.append(matching)

len(l_final), sma

(235, 10512)